# seq2seq

In [3]:
import gc
import sys
import os
import warnings
from tqdm import tqdm

sys.path.append(os.path.abspath("modules/"))
sys.path.append(os.path.abspath("fe_modules/"))
sys.path.append(os.path.abspath("seq2seq_modules/"))

os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')
tqdm.pandas()

In [4]:
import pandas as pd
import numpy as np
import time
import polars as pl
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn

from transformers import get_constant_schedule

In [5]:
import importlib

import modules
import fe_modules
import seq2seq_modules

importlib.reload(modules)
importlib.reload(fe_modules)
importlib.reload(seq2seq_modules)

from modules.memory_utils import pandas_reduce_mem_usage, pandas_string_to_cat, my_reset
from seq2seq_modules.preprocessing import PandasPreprocessor
from seq2seq_modules.models import LSTMModel
from seq2seq_modules.weight_initialization import weights_init_uniform_rule
from seq2seq_modules.loops import cross_validation, single_model_training
from seq2seq_modules.utils import age_bucket
from seq2seq_modules.metrics import AGE_METRIC

## Read and process

In [6]:
LOCAL_DATA_PATH = './data/'
SPLIT_SEED = 42

In [7]:
ids = ["user_id"]

cat_features = [
    "region_name",
    "city_name",
    "cpe_manufacturer_name",
    "cpe_model_name",
    "url_host",
    "cpe_type_cd",
    "cpe_model_os_type",
    "part_of_day",
    "request_cnt",
    "domain"
]

continous_features = [
    "price",
    "timestamp",
    "relative_date"
]

In [6]:
df = pandas_reduce_mem_usage(
    pd.read_parquet("seq2seq_data/version_1.parquet.gzip")
)
df.head()

Memory usage of dataframe is 13549.40 MB


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:03<00:00,  3.91it/s]

Memory usage after optimization is: 12317.64 MB
Decreased by 9.1%


,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,price,part_of_day,request_cnt,user_id,domain,timestamp,relative_date
0,21,409,1,589,5788,2,1,20368.0,2,1,45098,761,165525120.0,3888.0
1,21,409,1,589,12900,2,1,20368.0,2,1,45098,549,165559680.0,432.0
2,21,409,1,589,17626,2,1,20368.0,0,1,45098,712,165499200.0,6264.0
3,21,409,1,589,59366,2,1,20368.0,0,1,45098,712,165265920.0,29592.0
4,21,409,1,589,59366,2,1,20368.0,0,1,45098,712,165386880.0,17496.0


In [7]:
target = pandas_reduce_mem_usage(
    pd.read_parquet(
        f'{LOCAL_DATA_PATH}public_train.pqt', columns=["user_id", "age"]
    )
)
target

Memory usage of dataframe is 6.18 MB


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 478.31it/s]

Memory usage after optimization is: 3.60 MB
Decreased by 41.7%


,user_id,age
350459,350459,31.0
188276,188276,35.0
99002,99002,41.0
155506,155506,33.0
213873,213873,54.0
...,...,...
225374,225374,49.0
25776,25776,22.0
148131,148131,28.0
205570,205570,28.0


In [8]:
df = pandas_reduce_mem_usage(df.merge(target, how="left", on="user_id").dropna(subset=["age"]))
df.head()


KeyboardInterrupt



In [ ]:
df["age"] = df["age"].progress_apply(age_bucket)
df = pandas_reduce_mem_usage(df)
df.head()

In [ ]:
df.to_parquet("seq2seq_data/version_1_with_target.parquet.gzip",
              compression='gzip')

## Make torch Dataset

In [1]:
import gc
import sys
import os
import warnings
from tqdm import tqdm

sys.path.append(os.path.abspath("modules/"))
sys.path.append(os.path.abspath("fe_modules/"))
sys.path.append(os.path.abspath("seq2seq_modules/"))

os.environ['OPENBLAS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')
tqdm.pandas()

In [2]:
import pandas as pd
import numpy as np
import time
import polars as pl
import scipy
import implicit
import bisect
import sklearn.metrics as m
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

from sklearn.model_selection import train_test_split
from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn

from torch.optim.lr_scheduler import ReduceLROnPlateau
from transformers import get_constant_schedule, get_cosine_schedule_with_warmup

In [3]:
import importlib

import modules
import fe_modules
import seq2seq_modules

importlib.reload(modules)
importlib.reload(fe_modules)
importlib.reload(seq2seq_modules)

from modules.memory_utils import pandas_reduce_mem_usage, pandas_string_to_cat, my_reset
from seq2seq_modules.preprocessing import PandasPreprocessor
from seq2seq_modules.models import LSTMModel, StarterBERTModel, AttentionPoolingBERTModel
from seq2seq_modules.weight_initialization import weights_init_uniform_rule, weights_init_xavier
from seq2seq_modules.loops import cross_validation, single_model_training
from seq2seq_modules.utils import age_bucket
from seq2seq_modules.metrics import AGE_METRIC

In [4]:
LOCAL_DATA_PATH = './data/'
SPLIT_SEED = 42

In [5]:
ids = ["user_id"]

cat_features = [
    "region_name",
    "city_name",
    "cpe_manufacturer_name",
    "cpe_model_name",
    "url_host",
    "cpe_type_cd",
    "cpe_model_os_type",
    "part_of_day",
    "domain"
]

continous_features = [
    "request_cnt",
    "price",
    "timestamp",
    "relative_date"
]

In [6]:
df = pandas_reduce_mem_usage(
    pd.read_parquet("seq2seq_data/version_1_with_target.parquet.gzip")
)

Memory usage of dataframe is 8239.68 MB


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  6.08it/s]

Memory usage after optimization is: 8239.68 MB
Decreased by 0.0%


In [7]:
preprocessor = PandasPreprocessor(
         agg_column="user_id", 
         time_column="timestamp",
         target_column="age",
         features=cat_features + continous_features,
         max_len=1024,
         padding_side="right"
)

In [8]:
input_features, attention_masks, targets = preprocessor.transform(df)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 269998/269998 [00:00<00:00, 658956.53it/s]


In [9]:
dataset = TensorDataset(input_features, attention_masks, targets)

In [10]:
df[cat_features + continous_features].head()

,region_name,city_name,cpe_manufacturer_name,cpe_model_name,url_host,cpe_type_cd,cpe_model_os_type,part_of_day,domain,request_cnt,price,timestamp,relative_date
2272,60,732,27,211,5790,2,0,3,761,1,74259.0,163261440.0,10584.0
2273,60,732,27,211,65865,2,0,0,549,3,74259.0,162950400.0,41256.0
2274,60,732,27,211,111474,2,0,0,761,3,74259.0,162941760.0,42120.0
2275,14,311,27,211,111474,2,0,1,761,2,74259.0,163114560.0,24624.0
2276,60,732,27,211,125409,2,0,0,549,1,74259.0,163028160.0,33480.0


## Feed to the model

In [11]:
cat_feature_indexes = []
cont_feature_indexes = []
vocab_sizes = {}

for i in tqdm(range(len(cat_features))):
    cat_feature_indexes.append(i)
    vocab_sizes[i] = df[cat_features[i]].max() + 1

for i in tqdm(range(len(continous_features))):
    cont_feature_indexes.append(len(cat_features) + i)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 118149.41it/s]


In [12]:
vocab_sizes

{0: 81, 1: 985, 2: 37, 3: 599, 4: 199683, 5: 4, 6: 2, 7: 4, 8: 869}

In [16]:
# model = AttentionPoolingBERTModel(
#         cat_feature_indexes=cat_feature_indexes,
#         vocab_sizes=vocab_sizes,
#         cont_feature_indexes=cont_feature_indexes,
#         encoder_hidden_dim=16,
#         hidden_dim=256,
#         dim_feedforward=512,
#         output_dim=7,
#         pe_type="trainable",
#         use_mask=False,
#         max_len=1024,
#         use_key_padding_mask=True,
# )

# model = StarterBERTModel(
#         cat_feature_indexes=cat_feature_indexes,
#         vocab_sizes=vocab_sizes,
#         cont_feature_indexes=cont_feature_indexes,
#         encoder_hidden_dim=16,
#         hidden_dim=256,
#         dim_feedforward=512,
#         output_dim=7,
#         pe_type="trainable",
#         use_mask=False,
#         max_len=1024,
#         use_key_padding_mask=False,
#         starter="randn",
#         shared=True
# )

model = LSTMModel(
        cat_feature_indexes=cat_feature_indexes,
        vocab_sizes=vocab_sizes,
        cont_feature_indexes=cont_feature_indexes,
        encoder_hidden_dim=16,
        hidden_dim=256,
        output_dim=7,
)


weights_init_xavier(model)

loss = nn.CrossEntropyLoss(
    weight=torch.tensor(
        compute_class_weight(
            class_weight="balanced",
            classes=sorted(df["age"].unique()),
            y=targets.numpy()
        )
    )
)

metric = AGE_METRIC

In [17]:
cross_validation(
        project_name="ХУЙ", 
        model=model, 
        dataset=dataset, 
        loss_function=loss, 
        metric_func=AGE_METRIC, 
        optimizer=torch.optim.AdamW, 
        get_scheduler=get_cosine_schedule_with_warmup, 
        strat_array=targets.numpy(), 
        device='cuda', 
        random_state=69, 
        shuffle=True, 
        dataloader_shuffle=False, 
        n_folds=5, 
        epochs=2000, 
        lr=1e-3, 
        weight_decay=1e-4,
        start_fold=0, 
        batch_size=32,
)

FOLD 0
--------------------------------


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1688/1688 [00:38<00:00, 44.08it/s]


EPOCH: 0
{'Age F1': 0.18872538593025842, 'loss': 1.7745289922892868}
{'Age F1': 0.24184734356558005, 'loss': 1.7030532885677696}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1688/1688 [00:38<00:00, 43.52it/s]


EPOCH: 1
{'Age F1': 0.22986853977811794, 'loss': 1.6870437684113115}
{'Age F1': 0.25474874623782723, 'loss': 1.6511534026711905}


...

EPOCH: 7
{'Age F1': 0.3072666079591864, 'loss': 1.5093209619743022}
{'Age F1': 0.2923783582923893, 'loss': 1.5387942346612977}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1688/1688 [00:40<00:00, 42.09it/s]


EPOCH: 8
{'Age F1': 0.3148705951628563, 'loss': 1.4976822132091814}
{'Age F1': 0.29616781780325746, 'loss': 1.5434957649664283}


 25%|████████████████████████████████████████████▊                                                                                                                                         | 1660/6750 [01:36<04:54, 17.29it/s]


KeyboardInterrupt: 